In [ ]:
class Sentence_ent():
    
    def __init__(self):
        train_df = pd.read_csv('/Users/ebby/Downloads/SICK/SICK.txt', sep="\t", header=None)
        train_df.columns = train_df.iloc[0]
        train_df=train_df.reindex(train_df.index.drop(0))
        EMBEDDING_FILE = '/Users/ebby/Documents/Fake news/MALSTM/GoogleNews-vectors-negative300.bin'
        MODEL_SAVING_DIR = '/Users/ebby/Documents/Fake news/MALSTM'
        self.word2vec_location = '/Users/ebby/Documents/Fake news/MALSTM/GoogleNews-vectors-negative300.bin'
        
    def preprocess(self):
        stops = set(stopwords.words('english'))

        def text_to_word_list(text):
            ''' Pre process and convert texts to a list of words '''
            text = str(text)
            text = text.lower()

            # Clean the text
            text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
            text = re.sub(r"what's", "what is ", text)
            text = re.sub(r"\'s", " ", text)
            text = re.sub(r"\'ve", " have ", text)
            text = re.sub(r"can't", "cannot ", text)
            text = re.sub(r"n't", " not ", text)
            text = re.sub(r"i'm", "i am ", text)
            text = re.sub(r"\'re", " are ", text)
            text = re.sub(r"\'d", " would ", text)
            text = re.sub(r"\'ll", " will ", text)
            text = re.sub(r",", " ", text)
            text = re.sub(r"\.", " ", text)
            text = re.sub(r"!", " ! ", text)
            text = re.sub(r"\/", " ", text)
            text = re.sub(r"\^", " ^ ", text)
            text = re.sub(r"\+", " + ", text)
            text = re.sub(r"\-", " - ", text)
            text = re.sub(r"\=", " = ", text)
            text = re.sub(r"'", " ", text)
            text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
            text = re.sub(r":", " : ", text)
            text = re.sub(r" e g ", " eg ", text)
            text = re.sub(r" b g ", " bg ", text)
            text = re.sub(r" u s ", " american ", text)
            text = re.sub(r"\0s", "0", text)
            text = re.sub(r" 9 11 ", "911", text)
            text = re.sub(r"e - mail", "email", text)
            text = re.sub(r"j k", "jk", text)
            text = re.sub(r"\s{2,}", " ", text)

            text = text.split()

            return text

        # Prepare embedding
        vocabulary = dict()
        inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
        word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

        questions_cols = ['sentence_A' , 'sentence_B']

        # Iterate over the questions only of both training and test datasets
        for dataset in [self.test_df]:
            for index, row in dataset.iterrows():

                # Iterate through the text of both questions of the row
                for question in questions_cols:

                    q2n = []  # q2n -> question numbers representation
                    print(text_to_word_list(row[question]))
                    for word in text_to_word_list(row[question]):

                        # Check for unwanted words
                        if word in stops and word not in word2vec.vocab:
                            continue

                        if word not in vocabulary:
                            vocabulary[word] = len(inverse_vocabulary)
                            q2n.append(len(inverse_vocabulary))
                            inverse_vocabulary.append(word)
                            #print(word)
                        else:
                            q2n.append(vocabulary[word])

                    # Replace questions as word to question as number representation
                    self.dataset.set_value(index, question, q2n)

        self.embedding_dim = 300
        self.embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
        self.embeddings[0] = 0  # So that the padding will be ignored
        # Build the embedding matrix
        for word, index in vocabulary.items():
            if word in word2vec.vocab:
                self.embeddings[index] = word2vec.word_vec(word)

        del word2vec
        
        
        
    def pad_train_data_one(self):
        max_seq_length = max(train_df.sentence_A.map(lambda x: len(x)).max(),
                     train_df.sentence_B.map(lambda x: len(x)).max(),)

        X_train = self.train_df[questions_cols]
        Y_train = self.train_df['relatedness_score']
        # Split to dicts
        X_train = {'left': X_train.sentence_A, 'right': X_train.sentence_B}

        # Convert labels to their numpy representations
        Y_train = Y_train.values

        # Zero padding
        for dataset, side in itertools.product([X_train], ['left', 'right']):
            self.dataset[side] = pad_sequences(self.dataset[side], maxlen=max_seq_length)
        
        
    def load_dep(self) : 
        file = open(self.vocab_location,"rb")
        self.vocabulary = pickle.load(file)
        file.close()
        file = open(self.embeddings_location, "rb")
        self.embeddings = pickle.load(file)
        file.close()
        print("Dependencies loaded!")
        
    def to_index_and_pad(self) : 
        
        self.index_rep = []
        for word in self.word_list : 
            if word in self.vocabulary : 
                self.index_rep.append(vocabulary[word])
        self.index_rep = pad_sequences([self.index_rep], maxlen=26)
                

In [ ]:
class ENTAILMENT():
    
    def __init__(self) : 
        S = Sentence_ent()
        S.load_dep()
        self.sentence = S
        
        
    def load_model_malstm(self):
        n_hidden = 50
        gradient_clipping_norm = 1
        batch_size = 64
        n_epoch = 25

        def exponent_neg_manhattan_distance(left, right):

            ''' Helper function for the similarity estimate of the LSTMs outputs'''
            return (K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True)))*4.0 +1


        # The visible layer
        left_input = Input(shape=(max_seq_length,), dtype='int32')
        right_input = Input(shape=(max_seq_length,), dtype='int32')

        embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

        # Embedded version of the inputs
        encoded_left = embedding_layer(left_input)
        encoded_right = embedding_layer(right_input)

        # Since this is a siamese network, both sides share the same LSTM
        shared_lstm = LSTM(n_hidden)

        left_output = shared_lstm(encoded_left)
        right_output = shared_lstm(encoded_right)

        # Calculates the distance as defined by the MaLSTM model
        malstm_distance = Merge(mode=lambda x: ((K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True)))*4.0 +1), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

        # Pack it all up into a model
        self.malstm = Model([left_input, right_input], [malstm_distance])


        # Adadelta optimizer, with gradient clipping by norm
        optimizer = Adadelta(clipnorm=gradient_clipping_norm)

        self.malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

        # Start training
        training_start_time = time()

        self.malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch)

        print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

    def get_parameters(self,sentence_A_processed,sentence_B_processed):
        features_function = K.function([left_input,right_input], [left_output,right_output])
        features = features_function([sentence_A_processed,sentence_B_processed])
        np_feature_difference = np.array(features)
        left_matrix = np_feature_difference[0]
        right_matrix = np_feature_difference[1]
        diff_features= np.abs(np.subtract(left_matrix, right_matrix))
        prod = np.multiply(left_matrix,right_matrix)
        self.extracted_features = np.concatenate([diff_features,prod], axis = 1)
    
    def load_model_entailment(self):
        self.dense_model = keras.models.load_model('CONTRADICTION_PREDICTION.h5')
    
    def entailment_prediction(self):
        pred_val =self.dense_model.predict(self.extracted_features)
        print(pred_val)
        return pred_val        
        
    def predict_value(self, A, B):
#         "CONTRADICTION" : 0 , "ENTAILMENT" : 1 ,"NEUTRAL" : 2
        S = self.sentence
        
        S.sentence = A
        A = S.preprocess()
        S.sentence = B
        B = S.preprocess()        
        
        sentence_A_processed = A
        sentence_B_processed = B 
        self.load_model_malstm()
        self.get_parameters(sentence_A_processed,sentence_B_processed)
        self.load_model_entailment()
        entailment = self.entailment_prediction()
        return entailment

In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge,merge,Dense,Dropout
import keras.backend as K
from keras.optimizers import Adadelta,Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error,accuracy_score,confusion_matrix

Using TensorFlow backend.


In [66]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text


def PREDICT_CONTRADICTION(test):
    train_df = pd.read_csv('/Users/ebby/Downloads/SICK/SICK.txt', sep="\t", header=None)
    train_df.columns = train_df.iloc[0]
    train_df=train_df.reindex(train_df.index.drop(0))
    EMBEDDING_FILE = '/Users/ebby/Documents/Fake news/MALSTM/GoogleNews-vectors-negative300.bin'
    MODEL_SAVING_DIR = '/Users/ebby/Documents/Fake news/MALSTM'
    word2vec_location = '/Users/ebby/Documents/Fake news/MALSTM/GoogleNews-vectors-negative300.bin'
    stops = set(stopwords.words('english'))
    test_df = test#pd.read_csv('test file location', sep="\t", header=None)
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']  
    word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
    questions_cols = ['sentence_A' , 'sentence_B']

    for dataset in [train_df,test_df]:
        for index, row in dataset.iterrows():
            for question in questions_cols:

                q2n = []  
                for word in text_to_word_list(row[question]):

                    # Check for unwanted words
                    if word in stops and word not in word2vec.vocab:
                        continue

                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        q2n.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                        #print(word)
                    else:
                        q2n.append(vocabulary[word])
                dataset.set_value(index, question, q2n)

    embedding_dim = 300
    embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0 
    for word, index in vocabulary.items():
        if word in word2vec.vocab:
            embeddings[index] = word2vec.word_vec(word)

    del word2vec
    
    max_seq_length = max(train_df.sentence_A.map(lambda x: len(x)).max(),
                         train_df.sentence_B.map(lambda x: len(x)).max(),
                         test_df.sentence_A.map(lambda x: len(x)).max(),
                         test_df.sentence_B.map(lambda x: len(x)).max())

    X_train = train_df[questions_cols]
    Y_train = train_df['relatedness_score']
    X_test = test_df[questions_cols]
    X_train = {'left': X_train.sentence_A, 'right': X_train.sentence_B}
    X_test = {'left': test_df.sentence_A, 'right': test_df.sentence_B}
    Y_train = Y_train.values
    for dataset, side in itertools.product([X_train], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)
        
    X_train1 = train_df[questions_cols]
    Y_train1 = train_df['entailment_label']
    X_train1 = {'left': X_train1.sentence_A, 'right': X_train1.sentence_B}
    Y_train1 = Y_train1.values
    X_test1 = test_df[questions_cols]
    X_test1 = {'left': test_df.sentence_A, 'right': test_df.sentence_B}


    for dataset, side in itertools.product([X_train1,X_test1], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)
        
    n_hidden = 50
    gradient_clipping_norm = 1
    batch_size = 64
    n_epoch = 25

    def exponent_neg_manhattan_distance(left, right):
        return (K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True)))*4.0 +1

    left_input = Input(shape=(max_seq_length,), dtype='int32')
    right_input = Input(shape=(max_seq_length,), dtype='int32')
    embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)
    encoded_left = embedding_layer(left_input)
    encoded_right = embedding_layer(right_input)
    shared_lstm = LSTM(n_hidden)
    left_output = shared_lstm(encoded_left)
    right_output = shared_lstm(encoded_right)
    malstm_distance = Merge(mode=lambda x: ((K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True)))*4.0 +1), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])
    malstm = Model([left_input, right_input], [malstm_distance])
    optimizer = Adadelta(clipnorm=gradient_clipping_norm)
    malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
    training_start_time = time()
    malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch)
    print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

    features_function = K.function([left_input,right_input], [left_output,right_output])
    features = features_function([X_train1['left'],X_train1['right']])
    np_feature_difference = np.array(features)
    left_matrix = np_feature_difference[0]
    right_matrix = np_feature_difference[1]
    diff_features= np.abs(np.subtract(left_matrix, right_matrix))
    prod = np.multiply(left_matrix,right_matrix)
    extracted_features = np.concatenate([diff_features,prod], axis = 1)
    
    features = features_function([X_test1['left'],X_test1['right']])
    np_feature_difference = np.array(features)
    left_matrix = np_feature_difference[0]
    right_matrix = np_feature_difference[1]
    diff_features= np.abs(np.subtract(left_matrix, right_matrix))
    prod = np.multiply(left_matrix,right_matrix)
    extracted_features_test = np.concatenate([diff_features,prod], axis = 1)
    
    K.clear_session()
    model = Sequential()
    model.add(Dense(200, input_shape=(100,),kernel_initializer='he_normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, kernel_initializer='he_normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(Adam(lr = 0.001),'categorical_crossentropy', metrics=['accuracy'])
    y_TRAIN_cat = pd.get_dummies(Y_train1)
    model.fit(extracted_features, y_TRAIN_cat.values,batch_size=64,epochs=5,verbose=1)
    print("Entailmet model finished training")
    
    pred_val =model.predict_classes(extracted_features_test)
    
    return pred_val

In [67]:
pred =  PREDICT_CONTRADICTION(test)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:126: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:131: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/25
9840/9840 [==============================] - 18s 2ms/step - loss: 1.1110 - mean_squared_error: 1.1110
Epoch 2/25
9840/9840 [==============================] - 17s 2ms/step - loss: 0.4996 - mean_squared_error: 0.4996
Epoch 3/25
9840/9840 [==============================] - 17s 2ms/step - loss: 0.3902 - mean_squared_error: 0.3902
Epoch 4/25
9840/9840 [==============================] - 17s 2ms/step - loss: 0.3331 - mean_squared_error: 0.3331
Epoch 5/25
9840/9840 [==============================] - 17s 2ms/step - loss: 0.2992 - mean_squared_error: 0.2992
Epoch 6/25
9840/9840 [==============================] - 17s 2ms/step - loss: 0.2761 - mean_squared_error: 0.2761
Epoch 7/25
9840/9840 [==============================] - 17s 2ms/step - loss: 0.2595 - mean_squared_error: 0.2595
Epoch 8/25
9840/9840 [==============================] - 17s 2ms/step - loss: 0.2476 - mean_squared_error: 0.2476
Epoch 9/25
9840/9840 [==============================] - 432s 44ms/step - loss: 0.2377 - mean_squ

In [42]:
test = pd.DataFrame([["i am running away from home","i am flying to USA"],['The boy died in the truck' ,'The boy did not die in the truck'],['Today is Monday', 'This day is Monday'],["Assissins Creed will be releasing later this may","Assasins creed will not be releasing later this year"]])

In [43]:
test.columns =["sentence_A","sentence_B"]

In [68]:
pred

array([2, 2, 2, ..., 2, 2, 2])

In [71]:
x = pd.DataFrame(pred)


In [76]:
dictionary ={'NEUTRAL' : 2 , 'ENTAILMENT' : 1 , 'CONTRADICTION' :0}
test_df["3"] = test_df["3"].map(dictionary)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [77]:
val = test_df["3"]

In [77]:
test_df = pd.DataFrame(heading1,columns=["sentence_A"])

In [81]:
test_df["sentence_B"]=fake["title"]

In [83]:
news_dataset =pd.DataFrame()
#for loop here
#code here
crawled_data = pd.DataFrame(crawled_list,columns=["heading_crawled","article_crawled"])
crawled_data["fake_title"] = title_looped
#end_of one loop
news_dataset = news_dataset.append(crawled_data, ignore_index=True)


,sentence_A,sentence_B
0,hello,Muslims BUSTED: They Stole Millions In Gov’t B...
1,world,Re: Why Did Attorney General Loretta Lynch Ple...


In [ ]:
from __future__ import print_function
from functools import reduce
import json
import os
import re
import tarfile
import tempfile

import numpy as np
np.random.seed(1337)  # for reproducibility

'''
300D Model - Train / Test (epochs)
=-=-=
Batch size = 512
Fixed GloVe
- 300D SumRNN + Translate + 3 MLP (1.2 million parameters) - 0.8315 / 0.8235 / 0.8249 (22 epochs)
- 300D GRU + Translate + 3 MLP (1.7 million parameters) - 0.8431 / 0.8303 / 0.8233 (17 epochs)
- 300D LSTM + Translate + 3 MLP (1.9 million parameters) - 0.8551 / 0.8286 / 0.8229 (23 epochs)
Following Liu et al. 2016, I don't update the GloVe embeddings during training.
Unlike Liu et al. 2016, I don't initialize out of vocabulary embeddings randomly and instead leave them zeroed.
The jokingly named SumRNN (summation of word embeddings) is 10-11x faster than the GRU or LSTM.
Original numbers for sum / LSTM from Bowman et al. '15 and Bowman et al. '16
=-=-=
100D Sum + GloVe - 0.793 / 0.753
100D LSTM + GloVe - 0.848 / 0.776
300D LSTM + GloVe - 0.839 / 0.806
'''

import keras
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import merge, recurrent, Dense, Input, Dropout, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2
from keras.utils import np_utils

def extract_tokens_from_binary_parse(parse):
		return parse.replace('(', ' ').replace(')', ' ').replace('-LRB-', '(').replace('-RRB-', ')').split()

def yield_examples(fn, skip_no_majority=True, limit=None):
	for i, line in enumerate(open(fn)):
		if limit and i > limit:
			break
		data = json.loads(line)
		label = data['gold_label']
		s1 = ' '.join(extract_tokens_from_binary_parse(data['sentence1_binary_parse']))
		s2 = ' '.join(extract_tokens_from_binary_parse(data['sentence2_binary_parse']))
		if skip_no_majority and label == '-':
			continue
		yield (label, s1, s2)

def get_data(fn, limit=None):
	raw_data = list(yield_examples(fn=fn, limit=limit))
	left = [s1 for _, s1, s2 in raw_data]
	right = [s2 for _, s1, s2 in raw_data]
	print(max(len(x.split()) for x in left))
	print(max(len(x.split()) for x in right))

	LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
	Y = np.array([LABELS[l] for l, s1, s2 in raw_data])
	Y = np_utils.to_categorical(Y, len(LABELS))

	return left, right, Y

training = get_data('snli_1.0_train.jsonl')
validation = get_data('snli_1.0_dev.jsonl')
test = get_data('snli_1.0_test.jsonl')

tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(training[0] + training[1])

# Lowest index from the tokenizer is 1 - we need to include 0 in our vocab count
VOCAB = len(tokenizer.word_counts) + 1
LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
#RNN = recurrent.LSTM
#RNN = lambda *args, **kwargs: Bidirectional(recurrent.LSTM(*args, **kwargs))
#RNN = recurrent.GRU
#RNN = lambda *args, **kwargs: Bidirectional(recurrent.GRU(*args, **kwargs))
# Summation of word embeddings
RNN = None
LAYERS = 1
USE_GLOVE = True
TRAIN_EMBED = False
EMBED_HIDDEN_SIZE = 300
SENT_HIDDEN_SIZE = 300
BATCH_SIZE = 512
PATIENCE = 4 # 8
MAX_EPOCHS = 42
MAX_LEN = 42
DP = 0.2
L2 = 4e-6
ACTIVATION = 'relu'
OPTIMIZER = 'rmsprop'
print('RNN / Embed / Sent = {}, {}, {}'.format(RNN, EMBED_HIDDEN_SIZE, SENT_HIDDEN_SIZE))
print('GloVe / Trainable Word Embeddings = {}, {}'.format(USE_GLOVE, TRAIN_EMBED))

to_seq = lambda X: pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_LEN)
prepare_data = lambda data: (to_seq(data[0]), to_seq(data[1]), data[2])

training = prepare_data(training)
validation = prepare_data(validation)
test = prepare_data(test)

print('Build model...')
print('Vocab size =', VOCAB)

GLOVE_STORE = 'precomputed_glove.weights'
if USE_GLOVE:
	if not os.path.exists(GLOVE_STORE + '.npy'):
		print('Computing GloVe')
	
		embeddings_index = {}
		f = open('glove.840B.300d.txt')
		for line in f:
			values = line.split(' ')
			word = values[0]
			coefs = np.asarray(values[1:], dtype='float32')
			embeddings_index[word] = coefs
		f.close()
		
		# prepare embedding matrix
		embedding_matrix = np.zeros((VOCAB, EMBED_HIDDEN_SIZE))
		for word, i in tokenizer.word_index.items():
			embedding_vector = embeddings_index.get(word)
			if embedding_vector is not None:
				# words not found in embedding index will be all-zeros.
				embedding_matrix[i] = embedding_vector
			else:
				print('Missing from GloVe: {}'.format(word))
	
		np.save(GLOVE_STORE, embedding_matrix)

	print('Loading GloVe')
	embedding_matrix = np.load(GLOVE_STORE + '.npy')

	print('Total number of null word embeddings:')
	print(np.sum(np.sum(embedding_matrix, axis=1) == 0))

	embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable=TRAIN_EMBED)
else:
	embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, input_length=MAX_LEN)

rnn_kwargs = dict(output_dim=SENT_HIDDEN_SIZE, dropout_W=DP, dropout_U=DP)
SumEmbeddings = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=(SENT_HIDDEN_SIZE, ))

translate = TimeDistributed(Dense(SENT_HIDDEN_SIZE, activation=ACTIVATION))

premise = Input(shape=(MAX_LEN,), dtype='int32')
hypothesis = Input(shape=(MAX_LEN,), dtype='int32')

prem = embed(premise)
hypo = embed(hypothesis)

prem = translate(prem)
hypo = translate(hypo)

if RNN and LAYERS > 1:
	for l in range(LAYERS - 1):
		rnn = RNN(return_sequences=True, **rnn_kwargs)
		prem = BatchNormalization()(rnn(prem))
		hypo = BatchNormalization()(rnn(hypo))
rnn = SumEmbeddings if not RNN else RNN(return_sequences=False, **rnn_kwargs)
prem = rnn(prem)
hypo = rnn(hypo)
prem = BatchNormalization()(prem)
hypo = BatchNormalization()(hypo)

joint = merge([prem, hypo], mode='concat')
joint = Dropout(DP)(joint)
for i in range(3):
	joint = Dense(2 * SENT_HIDDEN_SIZE, activation=ACTIVATION, W_regularizer=l2(L2) if L2 else None)(joint)
	joint = Dropout(DP)(joint)
	joint = BatchNormalization()(joint)

pred = Dense(len(LABELS), activation='softmax')(joint)

model = Model(input=[premise, hypothesis], output=pred)
model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print('Training')
_, tmpfn = tempfile.mkstemp()
# Save the best model during validation and bail out of training early if we're not improving
callbacks = [EarlyStopping(patience=PATIENCE), ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)]
model.fit([training[0], training[1]], training[2], batch_size=BATCH_SIZE, nb_epoch=MAX_EPOCHS, validation_data=([validation[0], validation[1]], validation[2]), callbacks=callbacks)

# Restore the best found model during validation
model.load_weights(tmpfn)

loss, acc = model.evaluate([test[0], test[1]], test[2], batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import Input, Embedding, LSTM, Merge,merge,Dense,Dropout
import keras.backend as K
from keras.optimizers import Adadelta,Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error,accuracy_score,confusion_matrix

Using TensorFlow backend.


In [2]:
df = pd.read_csv('/Users/ebby/Downloads/SICK/SICK.txt', sep="\t", header=None)
df.columns = df.iloc[0]
df=df.reindex(df.index.drop(0))

In [3]:
train_df= df.loc[df['SemEval_set'].isin(['TRAIN','TRIAL'])]
test_df= df[df["SemEval_set"]== "TEST"]
EMBEDDING_FILE = '/Users/ebby/Documents/Fake news/MALSTM/GoogleNews-vectors-negative300.bin'
MODEL_SAVING_DIR = '/Users/ebby/Documents/Fake news/MALSTM'

In [4]:

stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

questions_cols = ['sentence_A' , 'sentence_B']

# Iterate over the questions only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, q2n)
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [5]:
max_seq_length = max(train_df.sentence_A.map(lambda x: len(x)).max(),
                     train_df.sentence_B.map(lambda x: len(x)).max(),
                     test_df.sentence_A.map(lambda x: len(x)).max(),
                     test_df.sentence_B.map(lambda x: len(x)).max())

X_train = train_df[questions_cols]
Y_train = train_df['relatedness_score']
X_test = test_df[questions_cols]
Y_test = test_df['relatedness_score']

# Split to dicts
X_train = {'left': X_train.sentence_A, 'right': X_train.sentence_B}
X_test = {'left': test_df.sentence_A, 'right': test_df.sentence_B}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_test = Y_test.values

# Zero padding
for dataset, side in itertools.product([X_train, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [6]:
X_train1 = train_df[questions_cols]
Y_train1 = train_df['entailment_label']
X_test1 = test_df[questions_cols]
Y_test1 = test_df['entailment_label']


# Split to dicts
X_train1 = {'left': X_train1.sentence_A, 'right': X_train1.sentence_B}
X_test1 = {'left': test_df.sentence_A, 'right': test_df.sentence_B}

# Convert labels to their numpy representations
Y_train1 = Y_train1.values
Y_test1 = Y_test1.values

# Zero padding
for dataset, side in itertools.product([X_train1, X_test1], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train1['left'].shape == X_train1['right'].shape
assert len(X_train1['left']) == len(Y_train1)

In [22]:
n_hidden = 50
gradient_clipping_norm = 1
batch_size = 64
n_epoch = 25

def exponent_neg_manhattan_distance(left, right):
    
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return (K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True)))*4.0 +1

def exponent_neg_manhattan_distance1(inputs):
    
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return (K.exp(-K.sum(inputs, axis=1, keepdims=True)))*4.0 +1


def Man_diff(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.abs((left-right))
def Man_prod(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.prod((left*right), axis=1, keepdims=True)

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])


# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

# Start training
training_start_time = time()

malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch)

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/25
4934/4934 [==============================] - 9s 2ms/step - loss: 1.5156 - mean_squared_error: 1.5156
Epoch 2/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.6498 - mean_squared_error: 0.6498
Epoch 3/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.4869 - mean_squared_error: 0.4869
Epoch 4/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.4050 - mean_squared_error: 0.4050
Epoch 5/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.3526 - mean_squared_error: 0.3526
Epoch 6/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.3169 - mean_squared_error: 0.3169
Epoch 7/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.2891 - mean_squared_error: 0.2891
Epoch 8/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.2674 - mean_squared_error: 0.2674
Epoch 9/25
4934/4934 [==============================] - 9s 2ms/step - loss: 0.2505 - mean_squared_error:

In [23]:
pred_truth =malstm.predict([X_test['left'],X_test['right']])

In [24]:
mean_squared_error(Y_test,pred_truth)

0.35345161647497408

In [25]:
intermediate_layer_model = Model(inputs=[malstm.input[0]], outputs=[malstm.layers[4].get_input_at(0)[0]])

In [26]:
intermediate_layer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 26, 300)           691200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                70200     
Total params: 761,400
Trainable params: 70,200
Non-trainable params: 691,200
_________________________________________________________________


In [41]:
K.clear_session()

In [42]:
left_matrix = intermediate_layer_model.predict([X_train1['left']])
right_matrix = intermediate_layer_model.predict(X_train['right'])
diff_features= np.abs(np.subtract(left_matrix, right_matrix))
prod = np.multiply(left_matrix,right_matrix)
extracted_features = np.concatenate([diff_features,prod], axis = 1)

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("input_1:0", shape=(?, 26), dtype=int32) is not an element of this graph.

In [30]:
left_matrix = intermediate_layer_model.predict([X_test1['left']])
right_matrix = intermediate_layer_model.predict(X_test1['right'])
diff_features= np.abs(np.subtract(left_matrix, right_matrix))
prod = np.multiply(left_matrix,right_matrix)
extracted_features_test = np.concatenate([diff_features,prod], axis = 1)

In [43]:
features_function = K.function([left_input,right_input], [left_output,right_output])
features = features_function([X_train1['left'],X_train1['right']])
np_feature_difference = np.array(features)
left_matrix = np_feature_difference[0]
right_matrix = np_feature_difference[1]
diff_features= np.abs(np.subtract(left_matrix, right_matrix))
prod = np.multiply(left_matrix,right_matrix)
extracted_features = np.concatenate([diff_features,prod], axis = 1)

ValueError: Tensor Tensor("lstm_1/TensorArrayReadV3:0", shape=(?, 50), dtype=float32) is not an element of this graph.

In [11]:
features = features_function([X_test1['left'],X_test1['right']])
np_feature_difference = np.array(features)
left_matrix = np_feature_difference[0]
right_matrix = np_feature_difference[1]
diff_features= np.abs(np.subtract(left_matrix, right_matrix))
prod = np.multiply(left_matrix,right_matrix)
extracted_features_val = np.concatenate([diff_features,prod], axis = 1)

In [35]:
K.clear_session()
model = Sequential()
model.add(Dense(200, input_shape=(100,),
                kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.compile(Adam(lr = 0.001),
              'categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
y_TRAIN_cat = pd.get_dummies(Y_train1)
y_TEST_cat = pd.get_dummies(Y_test1)

In [37]:
model.fit(extracted_features, y_TRAIN_cat.values, batch_size=16, epochs=5, verbose=1)

Epoch 1/5
4934/4934 [==============================] - 2s 313us/step - loss: 0.6588 - acc: 0.7284
Epoch 2/5
4934/4934 [==============================] - 1s 285us/step - loss: 0.4156 - acc: 0.8525
Epoch 3/5
4934/4934 [==============================] - 1s 286us/step - loss: 0.3912 - acc: 0.8561
Epoch 4/5
4934/4934 [==============================] - 1s 281us/step - loss: 0.3821 - acc: 0.8614
Epoch 5/5
4934/4934 [==============================] - 1s 281us/step - loss: 0.3755 - acc: 0.8650


In [38]:
pred_val =model.predict_classes(extracted_features_test)
dictionary = {"CONTRADICTION" : 0 , "ENTAILMENT" : 1 ,"NEUTRAL" : 2}
Y_test1_classes = test_df["entailment_label"].map(dictionary)
accuracy_score(pred_val,Y_test1_classes)

0.79800244598450876